In [3]:
!pip install flask-ngrok pyngrok
!ngrok authtoken 2jPvYS34ZllD1YvJmUA4rBQItRG_436Z1eM2nuB8aB173Fo8u  # You need to sign up for a free ngrok account to get a token
!ngrok config add-authtoken 2jPvYS34ZllD1YvJmUA4rBQItRG_436Z1eM2nuB8aB173Fo8u
!pip install flask-cors
!pip install --upgrade pyngrok flask-ngrok
!pip install requests

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
import numpy as np
import requests
import json
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
from sklearn.preprocessing import StandardScaler

def user_input_transformer(features):

    # Separate numerical and categorical features
    numerical_features = [
        features['package_weight(Kg)'],#
        features['Day'],#
        features['Month'],#
        features['Year'],#
        features['Price']
    ]

    categorical_features = [
        features['Produce_Variety'],#
        features['Commodity_Type'],#
        features['Unit']#
    ]

    # One-hot encode categorical features
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoded_categorical = encoder.fit_transform([categorical_features]).toarray()

    # Combine numerical and encoded categorical features
    my_data = np.concatenate((numerical_features, encoded_categorical.flatten()))

    # Reshape the input data to a 2D array
    input_data = np.array(my_data).reshape(1, -1)
    return input_data

In [6]:
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import joblib

app = Flask(__name__)

# HTML template for the input form
html_form = '''
<!DOCTYPE html>
<html>
<body>
    <h2>Agricultural Market Price Predictor</h2>
    <p>Enter the required information below:</p>
    <hr>
    <p>Note: All fields are required.</p>

    <hr>
    <form action="/predict" method="post">
        Produce Variety (E.g: Horticulture): <input type="text" name="Produce_Variety"><br>
        <br>
        Commodity Type (E.g: Cabbage): <input type="text" name="Commodity_Type"><br>
        <br>
        Unit (E.g: Ext. Box): <input type="text" name="Unit"><br>
        <br>
        Date (DD/MM/YYYY): <input type="text" name="Date"><br>
        <br>
        Package Weight in Kg (E.g: 120): <input type="number" name="package_weight(Kg)"><br>
        <br>
        Total Price of Commodities in Ksh (E.g: 2205): <input type="number" name="Price"><br>
        <br>
        <br>
        <br>
        <input type="submit" value="Predict">
    </form>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(html_form)

@app.route('/predict', methods=['POST'])
def predict():
  try:
    # Get the form data
    feature_values = request.form.to_dict()
    # load the trained model joblib file
    with open('base_model.sav', 'rb') as file:
      model = joblib.load(file)

    # Convert numeric fields
    numeric_fields = ['package_weight(Kg)', 'Price']
    for key in numeric_fields:
      feature_values[key] = float(feature_values[key])

    # Convert categorical fields
    categorical = ['Produce_Variety', 'Commodity_Type', 'Unit']
    for key in categorical:
        feature_values[key] = str(feature_values[key])

    # Convert the input date to a Unix timestamp
    date_obj = datetime.strptime(feature_values['Date'], '%d/%m/%Y')
    feature_values['Day'] = date_obj.day
    feature_values['Month'] = date_obj.month
    feature_values['Year'] = date_obj.year
    feature_values.pop('Date')

    input_data = user_input_transformer(feature_values)

    scaler_X = StandardScaler()
    input_data = scaler_X.fit_transform(input_data.reshape(1, -1))
    predicted_price = model.predict(input_data)

    return f"Predicted Price: {predicted_price[0]:.2f}"

  except Exception as e:
    return f"Error: {str(e)}",500
# Set up ngrok
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000/\"")

if __name__ == '__main__':
    app.run()

 * ngrok tunnel "NgrokTunnel: "https://500e-35-203-133-67.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 21:23:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 21:23:02] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.1 when using version 1.3.2. This might lead to breaking code or invalid results.